In [18]:
import numpy as np
import math
import cv2
import tdmclient.notebook
import utils

In [8]:
await tdmclient.notebook.start(tdm_addr='172.25.112.1', tdm_port=8596)

In [7]:
# await tdmclient.notebook.stop()

In [11]:
%%run_aseba
onevent run
    motor.left.target = 300
    motor.right.target = 300

onevent turnleft
    motor.left.target = -300
    motor.right.target = 300

onevent turnright
    motor.left.target = 300
    motor.right.target = -300

onevent stop
    motor.left.target = 0
    motor.right.target = 0

Define possible objects and marker colors

In [ ]:
robot1 = {"front":1, "back":2, "f_rgb": [136, 0, 20], "b_rgb": [3, 161, 234]}
robot2 = {"front":3, "back":4, "f_rgb": [23, 255, 22], "b_rgb": [163, 73, 164]}
ball = {"center":5, "rgb": [254, 242, 0]}

Define what robots are in the game

In [ ]:
robots = {
    "robot1": robot1,
    "robot2": robot2
}


Variables

In [19]:
COLOR_TOLERANCE = 20
DIST_BALL = 10

In [20]:
cap = cv2.VideoCapture(0)

[ WARN:0@1946.063] global cap_v4l.cpp:982 open VIDEOIO(V4L2:/dev/video0): can't open camera by index
[ERROR:0@1946.063] global obsensor_uvc_stream_channel.cpp:156 getStreamChannelGroup Camera index out of range


In [ ]:
while True:
    ret, frame = cap.read()
    if not ret:
        break
    